In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import IsolationForest

/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def read_txt_to_df(file_path):
    """
    Reads a text file and converts it into a pandas DataFrame.
    
    Args:
        file_path (str): Path to the text file.
    
    Returns:
        pd.DataFrame: DataFrame containing the data from the text file.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Extract header
    header = lines[0].strip().split('\t')
    
    # Extract data
    data = [line.strip().split('\t') for line in lines[1:]]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=header)
    
    return df

In [3]:
mrna= read_txt_to_df('data_mrna_seq_v2_rsem.txt')

In [4]:
mutations= read_txt_to_df('data_mutations.txt')

In [5]:
mrna

,Hugo_Symbol,Entrez_Gene_Id,TCGA-OR-A5J1-01,TCGA-OR-A5J2-01,TCGA-OR-A5J3-01,TCGA-OR-A5J5-01,TCGA-OR-A5J6-01,TCGA-OR-A5J7-01,TCGA-OR-A5J8-01,TCGA-OR-A5J9-01,...,TCGA-OR-A5LR-01,TCGA-OR-A5LS-01,TCGA-OR-A5LT-01,TCGA-OU-A5PI-01,TCGA-P6-A5OF-01,TCGA-PA-A5YG-01,TCGA-PK-A5H8-01,TCGA-PK-A5H9-01,TCGA-PK-A5HA-01,TCGA-PK-A5HB-01
0,NUDT18,79873,85.0144,165.884,309.288,178.931,124.066,283.594,283.465,123.753,...,154.812,176.182,177.721,157.925,524.443,316.228,317.302,232.051,303.749,159.984
1,SHC3,53358,97.5024,92.605,10.0726,49.574,5.5886,50.9426,36.0547,73.0957,...,46.0251,33.2035,75.6803,22.805,24.2472,113.731,46.8457,0.6979,192.944,50.5213
2,NCRNA00112,54089,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,FAM49B,51571,671.95,812.911,482.299,466.305,1074.68,594.465,938.666000000001,592.926,...,884.519,272.404,326.531,263.398,931.56,748.96,1241.1,784.437,897.464,718.925
4,FMO6P,388714,0,0,0,0,0,0,0.4144,0,...,0,0,0,0,0,0,0,0,0,0.401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZMAT5,55954,913.535,303.978,324.592,790.55,291.55,160.698,515.284,296.877,...,423.757,268.962,317.696,236.249,445.933,431.114,250.462,249.987,247.393,314.836
20527,AXIN2,8313,547.55,1793.32,522.589,337.723,9.5006,248.696,147.12,519.191,...,111.297,363.205,1063.78,376.853,418.85,611.65,62.461,45.7123,192.393,33.2799
20528,LONRF1,91694,363.112,592.672,456.821,484.895,628.155,228.239,276.005,427.355,...,919.665,254.108,1592.69,263.968,509.19,657.42,450.344,239.379,672.547,121.091
20529,KLHL10,317719,1.9212,0,0,1.5492,1.1177,0,0,0,...,0,0.6776,1.7007,0,0.3911,0.4623,0,1.0468,0.5513,0.401


In [6]:
mutations

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,SYMBOL_SOURCE,TREMBL,TSL,UNIPARC,VARIANT_CLASS,all_effects,cDNA_position,n_depth,t_depth,Annotation_Status
0,OPN4,94233,.,GRCh37,10,88419681,88419681,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI00001544ED,SNV,"OPN4,missense_variant,p.Gly277Asp,ENST00000241...",1090,140,143,SUCCESS
1,GOLGA3,2802,.,GRCh37,12,133360652,133360652,+,intron_variant,Intron,...,HGNC,.,.,UPI0000190979,SNV,"GOLGA3,missense_variant,p.Arg1122His,ENST00000...",.,16,11,SUCCESS
2,KLRB1,3820,.,GRCh37,12,9760409,9760409,+,missense_variant,Missense_Mutation,...,HGNC,.,.,UPI000006E273,SNV,"KLRB1,missense_variant,p.Glu9Asp,ENST000002294...",74,187,279,SUCCESS
3,SALL2,6297,.,GRCh37,14,21991730,21991730,+,missense_variant,Missense_Mutation,...,HGNC,F5H1G6_HUMAN,.,UPI00001AF54D,SNV,"SALL2,missense_variant,p.Arg574Gln,ENST0000045...",2427,79,101,SUCCESS
4,PLCB2,5330,.,GRCh37,15,40587141,40587141,+,synonymous_variant,Silent,...,HGNC,H0YNI4_HUMAN,.,UPI0000D79B75,SNV,"PLCB2,synonymous_variant,p.%3D,ENST00000260402...",2152,85,78,SUCCESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11655,SOX11,6664,.,GRCh37,2,5837544,5837544,+,3_prime_UTR_variant,3'UTR,...,HGNC,Q05CH0_HUMAN,.,UPI00001362AD,SNV,"SOX11,3_prime_UTR_variant,,ENST00000322002,;AC...",4746,69,41,SUCCESS
11656,NFXL1,152518,.,GRCh37,4,47850122,47850122,+,3_prime_UTR_variant,3'UTR,...,HGNC,.,.,UPI000020BC5D,SNV,"NFXL1,3_prime_UTR_variant,,ENST00000381538,;NF...",2971,15,17,SUCCESS
11657,PCDHB8,56128,.,GRCh37,5,140564475,140564475,+,downstream_gene_variant,3'Flank,...,HGNC,.,.,UPI000006D372,SNV,"PCDHB16,3_prime_UTR_variant,,ENST00000361016,;...",3496,89,66,SUCCESS
11658,C4B-AS1,106480730,.,GRCh37,6,32007391,32007391,+,upstream_gene_variant,5'Flank,...,HGNC,"Q9UP07_HUMAN,Q7KYP0_HUMAN,Q16874_HUMAN,C6K7H0_...",.,UPI0000073F52,SNV,"CYP21A2,synonymous_variant,p.%3D,ENST000004189...",776,315,437,SUCCESS
